In [30]:
"""Going to do nasa photos"""
#This project should be completed individually
# Pick a topic that is interesting to you.
# Topic: anything you choose, but it must include the following data types:

#     numeric features - Year, month, day (date), time of day
#     categorical features (to be dummy coded) - day of week, keywords, location
#     text features (to be processed using text analytics) - title, description, short_description, secondary_creator, photographer, center
#     image features (to be processed using image classification) - image
#     labels to choose from (i.e. outcomes that you want to predict with the other features) - labels

# There must be at least 500 records that include each of the data types below. But you could also have thousands. 
# However, there must be at least 500 records that include images. 
# So, for example, you might scrape 10000 tweets, but only 500 of them have images. That would be acceptable.

"""https://images.nasa.gov/docs/images.nasa.gov_api_docs.pdf - documentation should have everything I need"""

import pandas as pd
import requests, json


starsUrl3 = "https://images-api.nasa.gov/search?media_type=image&q=stars&page=3"
starsUrl4 = "https://images-api.nasa.gov/search?media_type=image&q=stars&page=4"
starsUrl5 = "https://images-api.nasa.gov/search?media_type=image&q=stars&page=5"
nebulaUrl = "https://images-api.nasa.gov/search?q=nebula&media_type=image&page=1"
nebulaUrl2 = "https://images-api.nasa.gov/search?q=nebula&media_type=image&page=2"
nebulaUrl3 = "https://images-api.nasa.gov/search?q=nebula&media_type=image&page=3"
jupiterUrl = "https://images-api.nasa.gov/search?q=jupiter&media_type=image&page=1"
jupiterUrl2 = "https://images-api.nasa.gov/search?q=jupiter&media_type=image&page=2"
jupiterUrl3 = "https://images-api.nasa.gov/search?q=jupiter&media_type=image&page=3"

urlList = [starsUrl3, starsUrl4, starsUrl5, nebulaUrl, nebulaUrl2, nebulaUrl3, jupiterUrl]

In [ ]:

df = pd.DataFrame(columns=['nasa_id', 'title', 'description', 'short_description', 'center', 'year_created', 'month_created', 'day_created', 'day_of_week_created', 'time_created', 'secondary_creator', 'keywords', 'image'])
pd.set_option('display.max_rows', 500)

import datetime
import calendar
 
def findDay(date):
    born = datetime.datetime.strptime(date, '%d %m %Y').weekday()
    return (calendar.day_name[born])

def addRows(urlIn, df, numIn):
  response = requests.request("GET", urlIn)
  index = numIn * 100
  itemsList = response.json()['collection']['items']
  dataDf = pd.DataFrame()
  linksDf = pd.DataFrame()
  for item in itemsList:

    # Add dates and time
    dateAndTime = item['data'][0]['date_created'].split('T')
    time = dateAndTime[1]
    date = dateAndTime[0]
    dates = date.split('-')
    df.at[index, 'year_created'] = dates[0]
    df.at[index, 'month_created'] = dates[1]
    df.at[index, 'day_created'] = dates[2]
    df.at[index, 'time_created'] = time
    dayOfWeek = dates[2] + ' ' + dates[1] + ' ' + dates[0]
    dayOfWeek = findDay(dayOfWeek)
    df.at[index, 'day_of_week_created'] = dayOfWeek

    '''Build new row first and then append it all'''
    # use try except blocks to handle situations where the json key doesn't exist
    try:
      df.at[index, 'keywords'] = item['data'][0]['keywords']
    except KeyError:
      pass

    try:
      df.at[index, 'title'] = item['data'][0]['title']
    except KeyError:
      pass

    try:
      df.at[index, 'nasa_id'] = item['data'][0]['nasa_id']
    except KeyError:
      pass

    try:
      df.at[index, 'description'] = item['data'][0]['description']
    except KeyError:
      pass

    try:
      df.at[index, 'short_description'] = item['data'][0]['description_508']
    except KeyError:
      pass
    
    try:
      df.at[index, 'center'] = item['data'][0]['center']

    except KeyError:
      pass

    try:
      df.at[index, 'secondary_creator'] = item['data'][0]['secondary_creator']
    except KeyError:
      pass

    try:
      df.at[index, 'image'] = item['links'][0]['href']
    except KeyError:
      pass  

    index += 1

for index, url in enumerate(urlList):
  print(url)
  addRows(url, df, index)


In [32]:
print(len(df.nasa_id.unique()))
print(len(df))
df.drop_duplicates('nasa_id',inplace=True)
len(df)
df

669
700


,nasa_id,title,description,short_description,center,year_created,month_created,day_created,day_of_week_created,time_created,secondary_creator,keywords,image
0,GSFC_20171208_Archive_e001677,Hubble Watches Star Clusters on a Collision Co...,"Image release August 16, 2012 Astronomers usi...",NaN,GSFC,2012,08,16,Thursday,18:40:59Z,NaN,[Hubble Watches Star Clusters on a Collision C...,https://images-assets.nasa.gov/image/GSFC_2017...
1,GRC-2019-C-09185,Evening With The Stars - 2019,Evening With The Stars - 2019,NaN,GRC,2019,09,10,Tuesday,00:00:00Z,NaN,"[Evening With The Stars, Lucas_9116-1, EWTS, X...",https://images-assets.nasa.gov/image/GRC-2019-...
2,GRC-2019-C-09186,Evening With The Stars - 2019,Evening With The Stars - 2019,NaN,GRC,2019,09,10,Tuesday,00:00:00Z,NaN,"[Evening With The Stars, Lucas_9116-1, EWTS, Z...",https://images-assets.nasa.gov/image/GRC-2019-...
3,GSFC_20171208_Archive_e000136,Star birth with a chance of Winds?,The lesser-known constellation of Canes Venati...,NaN,GSFC,2017,12,08,Friday,00:00:00Z,NaN,"[Star birth with a chance of Winds?, ngc4861]",https://images-assets.nasa.gov/image/GSFC_2017...
4,PIA01319,A Star Close Encounter,"The potential planet-forming disk (or ""protopl...",The potential planet-forming disk or protoplan...,JPL,2006,10,03,Tuesday,21:36:32Z,NASA/JPL-Caltech/Univ. of Ariz./Univ. of Szeged,"[Spitzer Space Telescope, IC 1396]",https://images-assets.nasa.gov/image/PIA01319/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,PIA01254,Hubble Provides Complete View of Jupiter Auroras,Hubble Provides Complete View of Jupiter Auroras,Hubble Provides Complete View of Jupiter Auroras,JPL,1998,05,02,Saturday,16:48:19Z,JPL/NASA/STScI,"[Jupiter, Hubble Space Telescope]",https://images-assets.nasa.gov/image/PIA01254/...
696,PIA01259,Hubble Views the Galileo Probe Entry Site on J...,Hubble Views the Galileo Probe Entry Site on J...,Hubble Views the Galileo Probe Entry Site on J...,JPL,1998,05,02,Saturday,16:51:12Z,NASA/ New Mexico State Univ.,"[Jupiter, Hubble Space Telescope]",https://images-assets.nasa.gov/image/PIA01259/...
697,PIA02883,"Jupiter Night-Side Auroras, North and South","Jupiter Night-Side Auroras, North and South","Jupiter Night-Side Auroras, North and South",JPL,2001,02,05,Monday,22:25:09Z,NASA/JPL/University of Arizona,"[Jupiter, Cassini-Huygens]",https://images-assets.nasa.gov/image/PIA02883/...
698,PIA02873,High Resolution Globe of Jupiter,This true-color simulated view of Jupiter is c...,This true-color simulated view of Jupiter is c...,JPL,2001,01,30,Tuesday,01:01:46Z,NASA/JPL/University of Arizona,"[Jupiter, Cassini-Huygens]",https://images-assets.nasa.gov/image/PIA02873/...


In [33]:
fileName = 'NasaImages.xlsx'
df.to_excel(fileName)